# Part 2 - Prompt Engineering

### Overview

This notebook demonstrates using the general-purpose GPT model for text classification.

The following steps are covered:
* Creating system prompt
* Using the Open AI ChatCompletion API
* Testing text classification

### Resources

* https://platform.openai.com/docs/guides/prompt-engineering
* https://platform.openai.com/docs/api-reference/chat


In [19]:
# %pip install openai --upgrade
# %pip show openai
# %pip install tiktoken

In [10]:
import pandas as pd
from openai import AsyncOpenAI
import os
import json
from src.util import distributionPreservingDownsample

In [12]:
from dotenv import load_dotenv; load_dotenv()
client = AsyncOpenAI(api_key=os.environ['OPENAI_API_KEY'])

# Check Token Requirements

Based on our expected completions, compute how many tokens are needed to encode those completions.  Then we set the `max_tokens` accordingly when using the API.

In [28]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-3.5-turbo-1106")

spamCompletion = 'Spam'
hamCompletion = 'Ham'

spamTokens = encoding.encode(spamCompletion)
hamTokens = encoding.encode(hamCompletion)

print(f'Spam token count: {len(spamTokens)}')
print(f'Ham token count: {len(hamTokens)}')

Spam token count: 2
Ham token count: 1


# Set Up API

In [45]:

async def isSpam(message, systemPrompts):
    messages = [{"role": "system", "content": systemPrompt} for systemPrompt in systemPrompts]
    messages.append({"role": "user", "content": message})
    response = await client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=messages,
        temperature=0.5,
        max_tokens=2
    )
    # print("Completion: {}".format(response.choices[0]))
    return response.choices[0].message.content == spamCompletion

# Test a Couple Examples

In [41]:
defaultSystemPrompt = "You will be provided with a text message. You will need to classify the text message as spam, ham. Spam is a text message that is spam, harmful, abusive, or otherwise unwanted. Ham is a text message that is not spam. Response should be a single word: Spam or Ham."

In [44]:
await isSpam("Hey what's happening? Want to get some ramen?", [defaultSystemPrompt])

False

In [46]:
await isSpam("Do you need help recovering your lost crypto tokens? We recover quickly with full privacy. Please send your private key and we will get started.", [defaultSystemPrompt])

True